In [20]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Semester 5/ML/ML Assignment 2/BitcoinHeistData.csv")

####Data Description

In [22]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier

df.info()
df.count()
print(df.label)
df = df.drop('address', axis=1)
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2916697 entries, 0 to 2916696
Data columns (total 10 columns):
 #   Column     Dtype  
---  ------     -----  
 0   address    object 
 1   year       int64  
 2   day        int64  
 3   length     int64  
 4   weight     float64
 5   count      int64  
 6   looped     int64  
 7   neighbors  int64  
 8   income     float64
 9   label      object 
dtypes: float64(2), int64(6), object(2)
memory usage: 222.5+ MB
0          princetonCerber
1           princetonLocky
2          princetonCerber
3          princetonCerber
4           princetonLocky
                ...       
2916692              white
2916693              white
2916694              white
2916695              white
2916696              white
Name: label, Length: 2916697, dtype: object


,year,day,length,weight,count,looped,neighbors,income,label
0,2017,11,18,0.008333,1,0,2,1.000500e+08,princetonCerber
1,2016,132,44,0.000244,1,0,1,1.000000e+08,princetonLocky
2,2016,246,0,1.000000,1,0,2,2.000000e+08,princetonCerber
3,2016,322,72,0.003906,1,0,2,7.120000e+07,princetonCerber
4,2016,238,144,0.072848,456,0,1,2.000000e+08,princetonLocky
...,...,...,...,...,...,...,...,...,...
2916692,2018,330,0,0.111111,1,0,1,1.255809e+09,white
2916693,2018,330,0,1.000000,1,0,1,4.409699e+07,white
2916694,2018,330,2,12.000000,6,6,35,2.398267e+09,white
2916695,2018,330,0,0.500000,1,0,1,1.780427e+08,white


####Train-Test-Validation set split

In [23]:
import math
def customTrainTestValidateSplit(df):
    n = df.label.count() #will give the size of the entire dataset
    trainnPoint = math.floor(n*0.7)
    testplusvalidaten = n - trainnPoint
    testplusvalidatePoint = trainnPoint + (math.floor(testplusvalidaten*0.5))
    shuffled_df = df.sample(frac = 1, random_state = 50)
    train_df = shuffled_df[:trainnPoint]
    test_df =  shuffled_df[trainnPoint:testplusvalidatePoint]
    validate_df = shuffled_df[testplusvalidatePoint:]
    return train_df, test_df, validate_df

train_df, test_df, validate_df = customTrainTestValidateSplit(df)
# print(train_df.count())
# print(test_df.count())
# print(validate_df.count())

#The print Value Show that the split is equal to the total count, hence the custom train-test-validate split works : 

valueOfSplits = train_df.label.count() + test_df.label.count() + validate_df.label.count()
mainDfCount = df.label.count()
print('The print value of the main count is', mainDfCount,', whereas the print value of the all splits is equal to',valueOfSplits)
if (valueOfSplits == mainDfCount):
    print("Eureka! It works")
else:
    print("Oh know!")



The print value of the main count is 2916697 , whereas the print value of the all splits is equal to 2916697
Eureka! It works


#Part A :

#####Dividing the train set into features and labels

In [24]:
from sklearn.metrics import classification_report
train_x_df = train_df.drop("label", axis=1)
train_y_df = train_df["label"].copy()

test_x_df = test_df.drop("label", axis=1)
test_y_df = test_df["label"].copy()

validate_x_df = validate_df.drop("label", axis=1)
validate_y_df = validate_df["label"].copy()

n = df.label.count()
maxDepthList = [4, 8, 10, 15, 20]
temporary = 0
test_df["label"]
i=df[df['label']=='white']
print(i)

         year  day  length     weight  count  looped  neighbors        income  \
41413    2011    1       0   1.000000      1       0          1  5.000000e+09   
41414    2011    1       0   1.000000      1       0          2  5.000000e+09   
41415    2011    1       0   1.000000      1       0          2  9.150000e+08   
41416    2011    1       0   1.000000      1       0          2  3.402500e+10   
41417    2011    1       0   1.000000      1       0          2  1.275100e+10   
...       ...  ...     ...        ...    ...     ...        ...           ...   
2916692  2018  330       0   0.111111      1       0          1  1.255809e+09   
2916693  2018  330       0   1.000000      1       0          1  4.409699e+07   
2916694  2018  330       2  12.000000      6       6         35  2.398267e+09   
2916695  2018  330       0   0.500000      1       0          1  1.780427e+08   
2916696  2018  330     144   0.073972   6800       0          2  1.123500e+08   

         label  
41413    w

####Decision Tree Using Gini Index

In [25]:
from sklearn.metrics import accuracy_score


for i in range(0,5):
    dtg = DecisionTreeClassifier(max_depth = maxDepthList[i]) #Gini is the default setting in this case
    y_pred = dtg.fit(train_x_df, train_y_df)
    y_pred = dtg.predict(test_x_df)
    y_predv = dtg.predict(validate_x_df)
    print("Accuracy ", (maxDepthList[i]) , "is", accuracy_score(test_y_df,y_pred), "(GINI INDEX) : Train")
    print("Accuracy ", (maxDepthList[i]) , "is", accuracy_score(validate_y_df,y_predv), "(GINI INDEX) : Validation")
    print()


Accuracy  4 is 0.9859635889875544 (GINI INDEX) : Train
Accuracy  4 is 0.9858081621924321 (GINI INDEX) : Validation

Accuracy  8 is 0.9865875818562073 (GINI INDEX) : Train
Accuracy  8 is 0.9864344407492486 (GINI INDEX) : Validation

Accuracy  10 is 0.9869190066399242 (GINI INDEX) : Train
Accuracy  10 is 0.9867018662643855 (GINI INDEX) : Validation

Accuracy  15 is 0.9882904195380624 (GINI INDEX) : Train
Accuracy  15 is 0.9881669923772299 (GINI INDEX) : Validation

Accuracy  20 is 0.9867978651672552 (GINI INDEX) : Train
Accuracy  20 is 0.9866790093827499 (GINI INDEX) : Validation



####Decision Tree Using entropy

In [26]:
for i in range(0,5):
    dtg = DecisionTreeClassifier(criterion='entropy',max_depth = maxDepthList[i])
    y_pred = dtg.fit(train_x_df, train_y_df).predict(test_x_df)
    print("Accuracy ", (maxDepthList[i]) , "is", accuracy_score(test_y_df,y_pred), "(ENTROPY) : Test")
    y_predv = dtg.predict(validate_x_df)
    print("Accuracy ", (maxDepthList[i]) , "is", accuracy_score(validate_y_df,y_predv), "(ENTROPY) : Validation")
    print()

#Entropy gives slightly (not a very significant amount of difference) better results than the gini index 
#https://quantdare.com/decision-trees-gini-vs-entropy/

Accuracy  4 is 0.9859635889875544 (ENTROPY) : Test
Accuracy  4 is 0.9858081621924321 (ENTROPY) : Validation

Accuracy  8 is 0.9862493000079999 (ENTROPY) : Test
Accuracy  8 is 0.9861190157826768 (ENTROPY) : Validation

Accuracy  10 is 0.9875909989600119 (ENTROPY) : Test
Accuracy  10 is 0.9873258591330385 (ENTROPY) : Validation

Accuracy  15 is 0.9882492771511183 (ENTROPY) : Test
Accuracy  15 is 0.9879635661306728 (ENTROPY) : Validation

Accuracy  20 is 0.9864778688243563 (ENTROPY) : Test
Accuracy  20 is 0.9863338704700517 (ENTROPY) : Validation



####Ensembling


In [27]:
hun = []
from sklearn.metrics import classification_report
import numpy as np
predictiabilityResult = []
predictiabilityResultv = [] #for validation set

def shufflingandselecting(df): #for choosing and shuffling different types of train_df at 50 percent of the sample
    shuffled_df = df.sample(frac = 1, random_state = 50)
    n = math.floor(df.label.count())
    df_split = np.array_split(df, 2)

    train_df = df_split[0]
    train_x_df = train_df.drop("label", axis=1)
    train_y_df = train_df["label"].copy()
    return train_x_df,train_y_df

for i in range(0,100):
    train_x_df,train_y_df = shufflingandselecting(train_df)
    s = "dt" + str(i)
    dtx = DecisionTreeClassifier(criterion='entropy', max_depth = 3)
    dtx.fit(train_x_df, train_y_df)
    y_pred = dtx.predict(test_x_df)
    predictiabilityResult.append(y_pred)
    y_predv = dtx.predict(validate_x_df)
    predictiabilityResultv.append(y_predv)

####Aggregation and predicting the final results

In [28]:
voting = pd.DataFrame(predictiabilityResult)
aggregationResult = voting.mode()
aggregationResult = aggregationResult.transpose()

votingv = pd.DataFrame(predictiabilityResultv)
aggregationResultv = voting.mode()
aggregationResultv = aggregationResultv.transpose()

In [31]:
print("Accuracy of aggregation function will be", accuracy_score(test_y_df,aggregationResult))
print("Accuracy of aggregation function will be", accuracy_score(validate_y_df,aggregationResultv),"Validation")

Accuracy of aggregation function will be 0.9859635889875544
Accuracy of aggregation function will be 0.9858081621924321 Validation


###Adaboost


In [32]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

dtg = DecisionTreeClassifier()

for i in range(0,5):
    abt = AdaBoostClassifier(base_estimator = dtg, n_estimators = maxDepthList[i], random_state=0)
    abt.fit(train_x_df, train_y_df)
    y_pred = abt.predict(test_x_df)
    y_predv = abt.predict(validate_x_df)
    print("Accuracy",(maxDepthList[i]),"is", accuracy_score(test_y_df,y_pred),"(DECISION TREE WITH ADABOOST) : Test")
    print("Accuracy",(maxDepthList[i]),"is", accuracy_score(validate_y_df,y_predv),"(DECISION TREE WITH ADABOOST) : Validation")
    print()

Accuracy 4 is 0.9849121724323151 (DECISION TREE WITH ADABOOST) : Test
Accuracy 4 is 0.9848893155506794 (DECISION TREE WITH ADABOOST) : Validation

Accuracy 8 is 0.9851247414315265 (DECISION TREE WITH ADABOOST) : Test
Accuracy 8 is 0.9849304579376236 (DECISION TREE WITH ADABOOST) : Validation

Accuracy 10 is 0.9837578999097153 (DECISION TREE WITH ADABOOST) : Test
Accuracy 10 is 0.9835224740288683 (DECISION TREE WITH ADABOOST) : Validation

Accuracy 15 is 0.9854538805270797 (DECISION TREE WITH ADABOOST) : Test
Accuracy 15 is 0.9853624530005372 (DECISION TREE WITH ADABOOST) : Validation

Accuracy 20 is 0.9865624392864082 (DECISION TREE WITH ADABOOST) : Test
Accuracy 20 is 0.9864275836847579 (DECISION TREE WITH ADABOOST) : Validation



####Using RF :

In [33]:
from sklearn.ensemble import RandomForestClassifier
for i in range(0,5):
    rf = RandomForestClassifier(n_estimators = maxDepthList[i])
    rf.fit(train_x_df, train_y_df)
    y_pred = rf.predict(test_x_df)
    y_predv = rf.predict(validate_x_df)
    print("Accuracy",(maxDepthList[i]),"is", accuracy_score(test_y_df,y_pred),"(RANDOM FOREST) : Test")
    print("Accuracy",(maxDepthList[i]),"is", accuracy_score(validate_y_df,y_predv),"(RANDOM FOREST) : Validation")
    print()

Accuracy 4 is 0.9846013188420704 (RANDOM FOREST) : Test
Accuracy 4 is 0.9844481777351116 (RANDOM FOREST) : Validation

Accuracy 8 is 0.9871818607787339 (RANDOM FOREST) : Test
Accuracy 8 is 0.9869350064570691 (RANDOM FOREST) : Validation

Accuracy 10 is 0.9875475708849042 (RANDOM FOREST) : Test
Accuracy 10 is 0.9873098593158935 (RANDOM FOREST) : Validation

Accuracy 15 is 0.9879178523674015 (RANDOM FOREST) : Test
Accuracy 15 is 0.9877075690563536 (RANDOM FOREST) : Validation

Accuracy 20 is 0.9880664220980332 (RANDOM FOREST) : Test
Accuracy 20 is 0.9877761397012605 (RANDOM FOREST) : Validation

